## The Monty Hall Problem

The Monty Hall problem arose from the gameshow <i>Let's Make a Deal</i>, where a guest had to choose which one of three doors had a prize behind it. The twist was that after the guest chose, the host, originally Monty Hall, would then open one of the doors the guest did not pick that also did not have the prize behind it. Afterwards, Monty would ask if the guest wanted to switch which door they had picked. Initial inspection may lead you to believe that if there are only two doors left there is a 50-50 chance of you picking the right one, and so there is no advantage one way or the other. However, it has been proven both through simulations and analytically that there is in fact a 66% chance of getting the prize if the guest switches their door after Monty opens one, regardless of the door they initially went with.

In [ ]:
%pylab inline
import seaborn; seaborn.set_style('whitegrid')
import torch

%load_ext watermark
%watermark -m -n -p torch,torchegranate

Populating the interactive namespace from numpy and matplotlib
torch        : 1.13.0
torchegranate: 0.4.0

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 4.15.0-206-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



We can reproduce this result in pomegranate using Bayesian networks with three nodes, one for the guest, one for the prize, and one for the door Monty chooses to open. The door the guest initially chooses and the door the prize is behind are completely random processes across the three doors, but the door which Monty opens is dependent on both the door the guest chooses (it cannot be the door the guest chooses), and the door the prize is behind (it cannot be the door with the prize behind it).

To create the Bayesian network in pomegranate, we first create the distributions which live in each node in the graph. For a categorical bayesian network we use Categorical distributions for the root nodes and ConditionalCategorical distributions for the inner and leaf nodes.

First, we can create our "prize" and "guest" distributions. These are each Categorical distributions because they do not depend on anything, and they are uniform distributions because they are equally likely to be any of the three doors.

In [ ]:
from torchegranate.distributions import Categorical

guest = Categorical([[1./3, 1./3, 1./3]])
prize = Categorical([[1./3, 1./3, 1./3]])

You may notice that there is an additional dimension added to the probabilities. This is because all distributions in pomegranate have the potential to be multivariate even when being applied to univariate problems.

Next, we need to create the conditional distribution describing the door that Monty will open. Because Monty can only open a door that is not selected by the contestant and also does not have the prize, sometimes this leaves Monty with only one door that can be opened. Overall, the distribution is a 3x3x3 tensor, with three possibilities from the guest, three independent possibilities from the prize, and three possible doors to open.

In [ ]:
from torchegranate.distributions import ConditionalCategorical

probs = numpy.array([[
     [[0.0, 0.5, 0.5], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]],
     [[0.0, 0.0, 1.0], [0.5, 0.0, 0.5], [1.0, 0.0, 0.0]],
     [[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.5, 0.5, 0.0]]
]])

monty = ConditionalCategorical(probs)

Next, we can create the Bayesian network object in just one line by passing in the distribution objects and edges in the form of (parent, child) tuples. Previous versions of pomegranate required that you create State or Node objects and add them in using `add_edge` and `add_node` methods. State and Node objects no longer exist, and while those methods still exist if you would prefer to use them you no longer need to. The `bake` method has also been removed and is no longer required.

In [15]:
from torchegranate.bayesian_network import BayesianNetwork

model = BayesianNetwork([guest, prize, monty], [(guest, monty), (prize, monty)])

In [16]:
X = torch.tensor([[0, 1, -1],
                  [0, 2, -1],
                  [2, 1, -1]])

X_masked = torch.masked.MaskedTensor(X, mask=X >= 0)


model.predict(X_masked)

<ipython-input-16-b464c03ef0f1>:5: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  X_masked = torch.masked.MaskedTensor(X, mask=X >= 0)


tensor([[0, 1, 2],
        [0, 2, 1],
        [2, 1, 0]])

In [20]:
from torchegranate.distributions import Exponential

X = torch.exp(torch.randn(100, 1))
mask = torch.ones(100, 1, dtype=torch.bool)
mask[75:] = False
X_masked = torch.masked.MaskedTensor(X, mask=mask)

Exponential().fit(X[:75]).scales

<ipython-input-20-79e1025ef2b7>:6: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  X_masked = torch.masked.MaskedTensor(X, mask=mask)


Parameter containing:
tensor([1.8173])

In [21]:
Exponential().fit(X_masked).scales

/usr/local/lib/python3.11/dist-packages/torch/masked/maskedtensor/core.py:102: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  return MaskedTensor(result_data, result_mask)
/usr/local/lib/python3.11/dist-packages/torchegranate/_utils.py:327: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  sample_weight = torch.masked.MaskedTensor(sample_weight,
/usr/local/lib/python3.11/dist-packages/torch/masked/maskedtensor/core.py:328: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our doc

Parameter containing:
tensor([1.8173])